## Q2, QDA ##

In [1]:
import numpy as np
import random
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt

In [123]:
def generate_n_i_range(K, n):
    cuts = np.zeros(K-1)
    for i in range(K-1):
        cuts[i] = random.randrange(0, n)
    n_i_range = np.sort(cuts)
    n_i_range = np.append(n_i_range,n)
    for i in range(K-1,0, -1):
        n_i_range[i] = int(n_i_range[i] - n_i_range[i-1])
    n_i_range = [int(x) for x in n_i_range]
    return n_i_range

def generate_data_QDA(K,p,n,mu_interval,cov_interval):
    
    matrix_singular = False
    while not matrix_singular:
        
        matrix_singular = False
        
        sigma = np.random.random((K, p, p))*(cov_interval[1] - cov_interval[0])+cov_interval[0]
        for i in range(K):
            
            sigma_i = sigma[i]
            sigma[i] = np.dot(sigma_i,sigma_i.transpose())
            
            if(np.linalg.det(sigma[i])):matrix_singular = True
        
        
        
    mu = np.random.random((K,p))*(mu_interval[1] - mu_interval[0]) + mu_interval[0]
    n_i_range = generate_n_i_range(K,n)
    pi = [x/n for x in n_i_range]
    x = np.zeros((n,p))
    
    for i in range(K):
        mu_i = mu[i,:]
        sigma_i = sigma[i,:,:]
        rv = multivariate_normal(mean = mu_i, cov = sigma_i)
        start_index = sum(n_i_range[:i])
        stop_index = sum(n_i_range[:i+1])
        generated_data = np.array(rv.rvs(size = n_i_range[i]))
        x[start_index:stop_index] = generated_data
    
    
    y_temp = np.zeros((n,K))
    
    
    for i in range(K):
        mu_i = mu[i,:]
        sigma_i = sigma[i,:,:]
        rv = multivariate_normal.pdf(x, mean = mu_i, cov = sigma_i)*pi[i]
        y_temp[:,i] = rv

    y = np.argmax(y_temp, 1)
    return x, y, pi, sigma, mu

def estimate_QDA(x,y):
    K = np.max(y) + 1
    n = x.shape[0]
    p = x.shape[1]

    pihat = np.zeros(K)
    muhat = np.zeros((K,p))
    sigmahat = np.zeros((K,p,p))
    for i in range(K):
        data_class = x[np.argwhere(y==i).reshape(-1), :]
        n_i = data_class.shape[0]
        
        pihat[i] = n_i/n
        
        muhat_i = 1/n_i*np.sum(data_class,0)
        muhat[i] = muhat_i 
        
        sigmahat_i = 1/(n_i-1)*np.dot((data_class - muhat_i).transpose(), data_class - muhat_i)
        sigmahat[i] = sigmahat_i
    
    return pihat,sigmahat,muhat


def plot_QDA(K, x_QDA, y_QDA):
    for i in range(K):
        plt.plot(x_QDA[y_QDA==i, 0], x_QDA[y_QDA==i, 1], '.')
    

## Q2, CART ##

In [124]:
import numpy as np
import random
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt

In [125]:
def generate_pi_range(K, n):
    cuts = np.zeros(K-1)
    for i in range(K-1):
        cuts[i] = random.randrange(0, n)
    pi_range = np.sort(cuts)
    pi_range = np.append(pi_range,n)
    for i in range(K-1,0, -1):
        pi_range[i] = int(pi_range[i] - pi_range[i-1])
    pi_range = [int(x) for x in pi_range]
    return pi_range

In [126]:
def split(limits):
    p = limits.shape[0]
    feat = int(random.randrange(0,p))
    place = np.random.random(1)*(limits[feat,0] - limits[feat,1])+ limits[feat,1]
    
    new_limits_high = np.copy(limits)
    new_limits_high[feat,0] = place
    
    new_limits_low = np.copy(limits)
    new_limits_low[feat,1] = place
    
    size_low = 1
    size_high = 1
    for i in range(p):
        size_low = size_low*(new_limits_low[i,1] - new_limits_low[i,0])
        size_high = size_high*(new_limits_high[i,1] - new_limits_high[i,0])
    
    return new_limits_low, new_limits_high, feat, place, size_low, size_high

In [127]:
def generate_data_CART(K ,p ,n ,nr_splits ,intervals):
    
    splits = np.zeros((nr_splits,2))
    all_limits = np.zeros((nr_splits + 1, p, 2))
    all_limits[0,:,:] = intervals
    sizes = np.zeros(nr_splits + 1)

    pointer = 1
    for i in range(nr_splits):
        j = int(random.randrange(0,pointer))
        base = all_limits[j,:,:]

        new_limits_low, new_limits_high, feat, place, size_low, size_high = split(base)
        splits[i,0] = place
        splits[i,1] = feat


        all_limits[j,:,:] = new_limits_low
        sizes[j] = size_low

        all_limits[pointer,:,:] = new_limits_high
        sizes[pointer] = size_high
        pointer += 1

        
    split_ranges = np.ones(nr_splits + 1) * (n / (nr_splits+1))
    split_ranges = np.multiply(split_ranges, sizes)
    split_ranges = split_ranges * (n/sum(split_ranges))
    

    data = np.zeros((n,p))
    classes = np.zeros((n,1))

    data_pointer = 0
    for k in range(nr_splits + 1):
        c = int(random.randrange(0,K))

        for i in range(int(split_ranges[k])+1):
            pnt = np.random.random(p)*(all_limits[k,:,0] - all_limits[k,:,1])+ all_limits[k,:,1]
            data[data_pointer + i,:] = pnt
            classes[data_pointer + i] = c

        data_pointer += int(split_ranges[k])

    return data, classes, splits

In [128]:
def plot_CART(p, data, classes):
    if(p == 2):
        """
        for i in range(nr_splits):
            if(int(splits[i,1]) == 0):
                splitlines = [np.ones(n)*splits[i][0], np.linspace(interval[0], interval[1], n)]
            else:
                splitlines = [np.linspace(interval[0], interval[1], n), np.ones(n)*splits[i][0]]

            plt.plot(splitlines[0], splitlines[1], color = 'black')
        """



        for i in range(K+1):
            indexes = np.argwhere(classes == i)[:,0]
            plt.plot(data[indexes,0], data[indexes,1],'.') 


        plt.xlim(interval)
        plt.ylim(interval)

    else:
        print("## NOTE: no plot since there are more than 2 features ##")

## Training ##

In [129]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [167]:
def train_and_test_both(K,p,n,nr_splits, interval, mu_interval, cov_interval, visualize = False):

    intervals = np.array([interval for i in range(p)])
    
    
    
    data_well_defined = False
    
    while not data_well_defined:
        x_CART, y_CART, splits = generate_data_CART(K ,p ,n ,nr_splits ,intervals)
        x_CART_train, x_CART_test, y_CART_train, y_CART_test = train_test_split(x_CART, y_CART, test_size=0.3, random_state=1)

    
        x_QDA, y_QDA,pi,sigma,mu = generate_data_QDA(K,p,n,mu_interval,cov_interval)
        x_QDA_train, x_QDA_test, y_QDA_train, y_QDA_test = train_test_split(x_QDA, y_QDA, test_size=0.3, random_state=1)
        
        
        CART_model = DecisionTreeClassifier()
        CART_model = CART_model.fit(x_CART_train,y_CART_train)
        y_CART_on_CART_pred = CART_model.predict(x_CART_test)

        
        data_well_defined = True
        try:
            CART_model = CART_model.fit(x_QDA_train,y_QDA_train)
            y_CART_on_QDA_pred = CART_model.predict(x_QDA_test)

            QDA_model = QuadraticDiscriminantAnalysis()
            QDA_model = QDA_model.fit(x_CART_train,y_CART_train)
            y_QDA_on_CART_pred = QDA_model.predict(x_CART_test)

            QDA_model = QDA_model.fit(x_QDA_train,y_QDA_train)
            y_QDA_on_QDA_pred = QDA_model.predict(x_QDA_test)
        except (ValueError, np.linalg.LinAlgError) as e:
            data_well_defined = False


    
    if(visualize):
        %matplotlib inline
        plt.rcParams["figure.figsize"]=15,6
        
        
        print(f'K = {K}')
        print(f'p = {p}')
        print(f'n = {n}')
        print(f'number of splits = {nr_splits}')
        
        
        print("Test error: CART on CART-data:", 1- metrics.accuracy_score(y_CART_test, y_CART_on_CART_pred))
        print("Test error: QDA on CART-data:",1 - metrics.accuracy_score(y_CART_test, y_QDA_on_CART_pred))
        
        plt.subplot(131)
        plot_CART(p, x_CART, y_CART)
        plt.title("Data simulated for CART")
        plt.xlim(interval)
        plt.ylim(interval)
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        
        plt.subplot(132)
        plot_CART(p, x_CART_test, y_CART_on_CART_pred)
        plt.title("CART on CART-data")
        plt.xlim(interval)
        plt.ylim(interval)
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        
        plt.subplot(133)
        plot_QDA(K, x_CART_test, y_QDA_on_CART_pred)
        plt.title("QDA on CART-data")
        plt.xlim(interval)
        plt.ylim(interval)
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        
        plt.savefig('saved_figure.png')
        plt.show()
        


        print("Test error: CART on QDA-data:",1 - metrics.accuracy_score(y_CART_test, y_CART_on_QDA_pred))
        print("Test error: QDA on QDA-data:",1 - metrics.accuracy_score(y_QDA_test, y_QDA_on_QDA_pred))
        
        
        plt.subplot(131)
        plot_QDA(K,x_QDA,y_QDA)
        plt.title("Data simulated for QDA")
        plt.xlim(interval)
        plt.ylim(interval)
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        
        plt.subplot(132)
        plot_CART(p, x_QDA_test, y_CART_on_QDA_pred)
        plt.title("CART on QDA-data")
        plt.xlim(interval)
        plt.ylim(interval)
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        
        
        plt.subplot(133)
        plot_QDA(K, x_QDA_test, y_QDA_on_QDA_pred)
        plt.title("QDA on QDA-data")
        plt.xlim(interval)
        plt.ylim(interval)
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        
        plt.savefig('saved_figure.png')
        plt.show()

    
    
    
    acc_CART_on_CART = metrics.accuracy_score(y_CART_test, y_CART_on_CART_pred)
    acc_QDA_on_CART = metrics.accuracy_score(y_CART_test, y_QDA_on_CART_pred)
    acc_CART_on_QDA = metrics.accuracy_score(y_QDA_test, y_CART_on_QDA_pred)
    acc_QDA_on_QDA = metrics.accuracy_score(y_QDA_test, y_QDA_on_QDA_pred)

    return 1-acc_CART_on_CART, 1-acc_QDA_on_CART, 1-acc_CART_on_QDA, 1-acc_QDA_on_QDA
    
           
    


In [168]:

number_runs = 10000
K = 7
p = 10
n = 1000
nr_splits = 10
interval = [-15.0, 15.0]


mu_interval = [-5, 5]
cov_interval = [-2, 2]


test_errors = np.zeros((number_runs, 4))


for i in range(number_runs):
    test_errors[i,:] = train_and_test_both(K,p,n,nr_splits, interval, mu_interval, cov_interval, visualize = False)
    


C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConver

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: User

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\fred

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: User

...

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:836: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel()

C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\fredr\Miniconda3\envs\big_data\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:

LinAlgError: singular matrix

In [ ]:
print(accuracies.shape)

print(f'mean accuracy CART on CART: {np.mean(test_errors[:,0])}')
print(f', with standard deviation: {np.std(test_errors[:,0])}')

print(f'mean accuracy QDA on CART: {np.mean(test_errors[:,1])}')
print(f', with standard deviation: {np.std(test_errors[:,1])}')

print(f'mean accuracy CART on QDA: {np.mean(test_errors[:,2])}')
print(f', with standard deviation: {np.std(test_errors[:,2])}')

print(f'mean accuracy QDA on QDA: {np.mean(test_errors[:,3])}')
print(f', with standard deviation: {np.std(test_errors[:,3])}')

plt.plot(test_errors[:,0])
plt.title("CART on CART")
plt.show()

plt.plot(test_errors[:,1])
plt.title("QDA on CART")
plt.show()

plt.plot(test_errors[:,2])
plt.title("CART on QDA")
plt.show()

plt.plot(test_errors[:,3])
plt.title("QDA on QDA")
plt.show()
